In [21]:
# Import the SparkSession class from the PySpark SQL module
from pyspark.sql import SparkSession

In [3]:
# Import specific functions from PySpark SQL module
# 'col' is used to access columns in a DataFrame for transformations
# 'sum' is imported as 'spark_sum' to avoid conflicts with Python's built-in 'sum' function
from pyspark.sql.functions import col, sum as spark_sum


In [4]:
# Initialize a SparkSession
spark = SparkSession.builder \
    .appName("Data Processing") \
    .getOrCreate()

24/12/12 22:29:20 WARN Utils: Your hostname, liltimz resolves to a loopback address: 127.0.1.1; using 172.29.249.245 instead (on interface eth0)
24/12/12 22:29:20 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/12 22:29:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [9]:
spark

In [5]:
# Downloading dataset using wget
# !wget https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2021-01.parquet


--2024-12-13 02:44:39--  https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2021-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.239.238.133, 18.239.238.152, 18.239.238.212, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.239.238.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 308924937 (295M) [application/x-www-form-urlencoded]
Saving to: ‘fhvhv_tripdata_2021-01.parquet’

fhvhv_tripdata_2021 100%[===================>] 294.61M  4.27MB/s    in 5m 9s   

2024-12-13 02:50:11 (976 KB/s) - ‘fhvhv_tripdata_2021-01.parquet’ saved [308924937/308924937]



In [6]:
ls

'1. using_MAX_date.png'*                 Untitled.ipynb*
 1.adding_indexes.png*                  'archive (1).zip'*
'2. using CTE.png'*                      dimension_model_starschema.png*
 Big_Data_Processing_with_Spark.ipynb*   fhvhv_tripdata_2021-01.parquet*
'ClickUp - clickup.csv.csv'*             question2.txt*
 ER_diagram_model.png*                   upload_source_datasets.ipynb*
 ETL_SCRIPTS.ipynb*                      validation_1.png*
'Float - allocations.csv.csv'*           validation_2.png*
'Global Health Statistics.csv'*          validation_3.png*
 LOAD_INTO_EDW.png*                      validation_4.png*
 LOAD_OLD.ipynb*                         validation_5.png*
'Query optimization question.png'*       validation_6.png*
'STARSCHEMA_AND ER_CODES.txt'*


In [11]:
# Read a Parquet file into a Spark DataFrame
df = spark.read \
    .option("header", "true") \
    .parquet('fhvhv_tripdata_2021-01.parquet')

In [12]:
df.show()

+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+----+----------+-------------------+-----------------+------------------+----------------+--------------+
|hvfhs_license_num|dispatching_base_num|originating_base_num|   request_datetime|  on_scene_datetime|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|trip_miles|trip_time|base_passenger_fare|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|shared_request_flag|shared_match_flag|access_a_ride_flag|wav_request_flag|wav_match_flag|
+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+--

In [13]:
df.schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('originating_base_num', StringType(), True), StructField('request_datetime', TimestampNTZType(), True), StructField('on_scene_datetime', TimestampNTZType(), True), StructField('pickup_datetime', TimestampNTZType(), True), StructField('dropoff_datetime', TimestampNTZType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('trip_miles', DoubleType(), True), StructField('trip_time', LongType(), True), StructField('base_passenger_fare', DoubleType(), True), StructField('tolls', DoubleType(), True), StructField('bcf', DoubleType(), True), StructField('sales_tax', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True), StructField('airport_fee', DoubleType(), True), StructField('tips', DoubleType(), True), StructField('driver_pay', DoubleType(), True), StructField('shared_re

In [14]:
# Extract the first 1001 lines from the Parquet file and save it to a new file named head.parquet
!head -n 1001 fhvhv_tripdata_2021-01.parquet > head.parquet

In [15]:
ls

'1. using_MAX_date.png'*                 Untitled.ipynb*
 1.adding_indexes.png*                  'archive (1).zip'*
'2. using CTE.png'*                      dimension_model_starschema.png*
 Big_Data_Processing_with_Spark.ipynb*   fhvhv_tripdata_2021-01.parquet*
'ClickUp - clickup.csv.csv'*             head.parquet*
 ER_diagram_model.png*                   question2.txt*
 ETL_SCRIPTS.ipynb*                      upload_source_datasets.ipynb*
'Float - allocations.csv.csv'*           validation_1.png*
'Global Health Statistics.csv'*          validation_2.png*
 LOAD_INTO_EDW.png*                      validation_3.png*
 LOAD_OLD.ipynb*                         validation_4.png*
'Query optimization question.png'*       validation_5.png*
'STARSCHEMA_AND ER_CODES.txt'*           validation_6.png*


In [16]:
# Repartition the DataFrame into 24 partitions
# Repartitioning redistributes the data across a specified number of partitions
# This is useful for parallel processing in distributed systems like Spark
# Increasing partitions can improve performance for large datasets by leveraging more resources
# However, excessive repartitioning can lead to overhead and reduced performance
df = df.repartition(24)

In [17]:
# Print the schema of the DataFrame
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- originating_base_num: string (nullable = true)
 |-- request_datetime: timestamp_ntz (nullable = true)
 |-- on_scene_datetime: timestamp_ntz (nullable = true)
 |-- pickup_datetime: timestamp_ntz (nullable = true)
 |-- dropoff_datetime: timestamp_ntz (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- trip_miles: double (nullable = true)
 |-- trip_time: long (nullable = true)
 |-- base_passenger_fare: double (nullable = true)
 |-- tolls: double (nullable = true)
 |-- bcf: double (nullable = true)
 |-- sales_tax: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)
 |-- tips: double (nullable = true)
 |-- driver_pay: double (nullable = true)
 |-- shared_request_flag: string (nullable = true)
 |-- shared_match_flag: string (nullable = true)
 |-- access_a_ride_f

In [18]:
# Import the 'types' module from PySpark SQL
from pyspark.sql import types

In [19]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- originating_base_num: string (nullable = true)
 |-- request_datetime: timestamp_ntz (nullable = true)
 |-- on_scene_datetime: timestamp_ntz (nullable = true)
 |-- pickup_datetime: timestamp_ntz (nullable = true)
 |-- dropoff_datetime: timestamp_ntz (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- trip_miles: double (nullable = true)
 |-- trip_time: long (nullable = true)
 |-- base_passenger_fare: double (nullable = true)
 |-- tolls: double (nullable = true)
 |-- bcf: double (nullable = true)
 |-- sales_tax: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)
 |-- tips: double (nullable = true)
 |-- driver_pay: double (nullable = true)
 |-- shared_request_flag: string (nullable = true)
 |-- shared_match_flag: string (nullable = true)
 |-- access_a_ride_f

In [20]:
df.show()

+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+----+----------+-------------------+-----------------+------------------+----------------+--------------+
|hvfhs_license_num|dispatching_base_num|originating_base_num|   request_datetime|  on_scene_datetime|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|trip_miles|trip_time|base_passenger_fare|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|shared_request_flag|shared_match_flag|access_a_ride_flag|wav_request_flag|wav_match_flag|
+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+--

In [23]:
# Import PySpark SQL functions module with an alias 'F'
from pyspark.sql import functions as F

In [24]:
# Transform the DataFrame by adding new columns and selecting specific fields

df \
    # Create a new column 'pickup_date' by extracting the date from 'pickup_datetime'
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    
    # Create a new column 'dropoff_date' by extracting the date from 'dropoff_datetime'
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    
    # Select specific columns for further processing or inspection
    .select('pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID') \
    
    # Show the first 20 rows of the resulting DataFrame
    .show()


+-----------+------------+------------+------------+
|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-----------+------------+------------+------------+
| 2021-01-30|  2021-01-30|          74|          42|
| 2021-01-23|  2021-01-23|          75|          51|
| 2021-01-20|  2021-01-20|         234|         161|
| 2021-01-20|  2021-01-20|         246|         116|
| 2021-01-16|  2021-01-16|          18|         247|
| 2021-01-26|  2021-01-26|          26|          26|
| 2021-01-27|  2021-01-27|         231|          87|
| 2021-01-23|  2021-01-23|         229|         148|
| 2021-01-22|  2021-01-23|         222|          76|
| 2021-01-09|  2021-01-09|          70|           7|
| 2021-01-31|  2021-01-31|         219|         191|
| 2021-01-11|  2021-01-11|         216|         225|
| 2021-01-09|  2021-01-09|          49|         164|
| 2021-01-15|  2021-01-15|         107|         158|
| 2021-01-31|  2021-01-31|          61|          36|
| 2021-01-26|  2021-01-26|         203|       

In [25]:
# Perform the aggregation: Sum miles traveled by pickup location
aggregated_df = df.groupBy("PULocationID").agg(
    spark_sum("trip_miles").alias("total_miles_traveled")
)

In [26]:
# Sort the results in descending order of total miles
aggregated_df = aggregated_df.orderBy(col("total_miles_traveled").desc())

In [27]:
# Show the aggregated results
aggregated_df.show()


+------------+--------------------+
|PULocationID|total_miles_traveled|
+------------+--------------------+
|         132|  1557488.3880000005|
|         138|   820273.3120000021|
|          61|   811149.6109999974|
|          76|   759131.2820000019|
|          42|   673048.3449999972|
|         244|    642341.228999999|
|          37|   597753.1059999964|
|          39|   523929.1189999995|
|          74|   507618.4289999965|
|          89|  506878.12600000115|
|         225|   493308.9000000046|
|           7|   479864.4870000013|
|          75|  475414.34199999925|
|          79|   471258.2480000017|
|          17|   471169.6469999993|
|         188|  456665.89599999774|
|         231|   442949.4580000007|
|          35|  435490.47300000134|
|         168|   435432.8990000009|
|          48|  429663.33100000105|
+------------+--------------------+
only showing top 20 rows



In [28]:
# Aggregating revenue components by pickup location
revenue_df = df.groupBy("PULocationID").agg(
    spark_sum("base_passenger_fare").alias("total_fare"),
    spark_sum("tips").alias("total_tips"),
    spark_sum("tolls").alias("total_tolls"),
    (spark_sum("base_passenger_fare") + spark_sum("tips") + spark_sum("tolls")).alias("total_revenue")
)

In [29]:
# Order results by total revenue in descending order
revenue_df = revenue_df.orderBy(col("total_revenue").desc())

In [30]:
# Show the result
revenue_df.show()

+------------+------------------+------------------+------------------+------------------+
|PULocationID|        total_fare|        total_tips|       total_tolls|     total_revenue|
+------------+------------------+------------------+------------------+------------------+
|         132| 4421060.100000002|150707.65999999995|266604.16000000294|4838371.9200000055|
|          61|3549191.7400000263| 83230.73000000005|  56794.3299999998| 3689216.800000026|
|         138|2623634.6199999996|         114626.52|246236.33000000185|2984497.4700000016|
|          76|2838696.6900000074| 25652.32000000001| 50450.76999999974| 2914799.780000007|
|          37|2586615.2200000174| 69947.91000000003|  71450.3799999996| 2728013.510000017|
|          42|  2509780.78000004| 55064.50000000004|108541.53999999938|2673386.8200000394|
|         244| 2233489.120000002| 67305.06000000001|125807.67999999982|2426601.8600000017|
|         225|2177129.1400000183| 52442.77000000001| 47570.77999999983| 2277142.690000018|

In [31]:

# Save the results to Parquet
revenue_df.write.mode("overwrite").parquet("output/revenue_by_pickup_location.parquet")

In [32]:
# Calculate average trip distance and time by base
avg_trip_metrics_df = df.groupBy("dispatching_base_num").agg(
    F.avg("trip_miles").alias("avg_trip_distance"),
    F.avg("trip_time").alias("avg_trip_time")
)

# Show the result
avg_trip_metrics_df.show()

# Save the results to Parquet
avg_trip_metrics_df.write.mode("overwrite").parquet("output/avg_trip_metrics_by_base.parquet")


+--------------------+------------------+------------------+
|dispatching_base_num| avg_trip_distance|     avg_trip_time|
+--------------------+------------------+------------------+
|              B02876| 4.535434313402564| 947.8165750741119|
|              B03136|3.8182075976457996| 939.3638309256287|
|              B02877| 4.479920138191076| 936.3618615601046|
|              B02869| 4.557642524408418|   956.82496051741|
|              B02883| 4.549834271641001| 951.7772652585221|
|              B02835| 4.444018058352365| 931.5443888691793|
|              B02884| 4.490663047106027| 945.1042402415455|
|              B02880| 4.555735023872853| 950.5938341738481|
|              B02878| 4.384423693884554| 928.8864149891191|
|              B02836|4.4604985573326665| 938.0215723292844|
|              B02872| 4.417193889465468| 944.6387043764055|
|              B02512| 4.473853858153785| 964.5206981528792|
|              B02867| 4.517315830378619| 952.9899010802533|
|              B02866|4.

In [33]:
# Count trips by drop-off location
dropoff_count_df = df.groupBy("DOLocationID").count().alias("trip_count")

# Order by trip count in descending order
dropoff_count_df = dropoff_count_df.orderBy(col("count").desc())

# Show the top 10 drop-off locations
dropoff_count_df.show(10)


+------------+------+
|DOLocationID| count|
+------------+------+
|         265|369502|
|          61|218654|
|          76|181207|
|          37|148401|
|          42|144159|
|         244|128205|
|          17|126426|
|         225|125420|
|         188|121635|
|          89|120126|
+------------+------+
only showing top 10 rows



In [34]:
# Extract hour from pickup datetime
hourly_trips_df = df.withColumn("hour", F.hour("pickup_datetime"))

# Count trips by hour
hourly_trip_counts = hourly_trips_df.groupBy("hour").count().alias("trip_count")

# Order by hour
hourly_trip_counts = hourly_trip_counts.orderBy(col("hour"))

# Show the result
hourly_trip_counts.show()

+----+------+
|hour| count|
+----+------+
|   0|361590|
|   1|267724|
|   2|196304|
|   3|150020|
|   4|135427|
|   5|188020|
|   6|317002|
|   7|467825|
|   8|587343|
|   9|545605|
|  10|528413|
|  11|536807|
|  12|572737|
|  13|618757|
|  14|674211|
|  15|698975|
|  16|708869|
|  17|765136|
|  18|765787|
|  19|711875|
+----+------+
only showing top 20 rows



In [35]:
# This can improve parallelism and performance for large-scale transformations
df = df.repartition(24)

In [36]:
# Verify the new number of partitions
print(f"Number of partitions after repartitioning: {df.rdd.getNumPartitions()}")

Number of partitions after repartitioning: 24


In [37]:
# Perform a transformation (e.g., aggregating total miles by pickup location)
aggregated_df = df.groupBy("PULocationID").agg(
    spark_sum("trip_miles").alias("total_miles_traveled")
)

In [38]:
# Calculate revenue per mile for each pickup location
pricing_efficiency_df = df.withColumn("total_revenue", F.col("base_passenger_fare") + F.col("tips") + F.col("tolls")) \
    .groupBy("PULocationID").agg(
        (F.sum("total_revenue") / F.sum("trip_miles")).alias("revenue_per_mile")
    )

# Order by revenue per mile in descending order
pricing_efficiency_df = pricing_efficiency_df.orderBy(col("revenue_per_mile").desc())

# Show the result
pricing_efficiency_df.show()


+------------+------------------+
|PULocationID|  revenue_per_mile|
+------------+------------------+
|         105| 5.970031545741325|
|         158| 5.162613742959451|
|         211| 5.137757542715089|
|         249| 5.107520722462131|
|         113| 5.065563055110756|
|         114| 5.015485077531107|
|         234| 4.989990119586515|
|         237| 4.938703931272939|
|         125|4.9086389504198715|
|         161|  4.90826174946643|
|          90|4.8992049671870115|
|         144| 4.896951486930965|
|         246| 4.865361593947758|
|         217| 4.841148664717978|
|          43| 4.813942657132882|
|         194| 4.798026733981097|
|         255| 4.771493134776462|
|         189| 4.763575819408854|
|          26| 4.757514567269955|
|         163| 4.755057410109206|
+------------+------------------+
only showing top 20 rows



In [39]:
# End the Spark session
spark.stop()